# Ejemplos Prácticos de Pytest para Data Engineering

En este notebook, exploraremos ejemplos prácticos y casos de uso específicos de pytest en el contexto de Data Engineering. Estos ejemplos están diseñados para mostrar cómo pytest puede ayudarnos a resolver problemas reales que enfrentamos en proyectos de ingeniería de datos.

Utilizaremos nuestro dataset de ventas de productos y crearemos ejemplos que cubren diferentes aspectos del testing en Data Engineering, desde la validación de datos hasta el testing de pipelines de ETL.

## Configuración Inicial

Primero, vamos a importar las bibliotecas necesarias y cargar nuestro dataset:

In [ ]:
import pandas as pd
import numpy as np
import pytest
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns

# Añadimos el directorio raíz al path para poder importar los módulos
sys.path.append(os.path.abspath('..'))

# Cargamos el dataset
df_ventas = pd.read_csv('../data/ventas_productos.csv')

# Mostramos las primeras filas
df_ventas.head()

## Ejemplo 1: Testing de Funciones de Agregación

En Data Engineering, a menudo necesitamos agregar datos para análisis y reportes. Vamos a crear una función que calcule las ventas totales por categoría y mes, y luego escribiremos tests para verificar su correcto funcionamiento.

In [ ]:
def agregar_ventas_por_categoria_y_mes(df):
    """Agrega las ventas por categoría y mes.
    
    Args:
        df: DataFrame con columnas 'fecha', 'categoria' y 'total'
        
    Returns:
        DataFrame: DataFrame con ventas agregadas por categoría y mes
    """
    # Convertimos la columna fecha a datetime
    df = df.copy()
    df['fecha'] = pd.to_datetime(df['fecha'])
    
    # Extraemos el mes y año
    df['mes'] = df['fecha'].dt.strftime('%Y-%m')
    
    # Agregamos por categoría y mes
    ventas_agregadas = df.groupby(['categoria', 'mes'])['total'].sum().reset_index()
    
    # Ordenamos por categoría y mes
    ventas_agregadas = ventas_agregadas.sort_values(['categoria', 'mes'])
    
    return ventas_agregadas

# Probamos la función
ventas_por_categoria_y_mes = agregar_ventas_por_categoria_y_mes(df_ventas)
ventas_por_categoria_y_mes.head(10)

Ahora, vamos a escribir tests para esta función. Crearemos un archivo temporal para los tests:

In [ ]:
%%file ../tests/test_agregacion.py
import pytest
import pandas as pd
import numpy as np

def agregar_ventas_por_categoria_y_mes(df):
    """Agrega las ventas por categoría y mes."""
    # Convertimos la columna fecha a datetime
    df = df.copy()
    df['fecha'] = pd.to_datetime(df['fecha'])
    
    # Extraemos el mes y año
    df['mes'] = df['fecha'].dt.strftime('%Y-%m')
    
    # Agregamos por categoría y mes
    ventas_agregadas = df.groupby(['categoria', 'mes'])['total'].sum().reset_index()
    
    # Ordenamos por categoría y mes
    ventas_agregadas = ventas_agregadas.sort_values(['categoria', 'mes'])
    
    return ventas_agregadas

@pytest.fixture
def df_ventas_test():
    """Fixture que crea un DataFrame de prueba."""
    data = {
        'fecha': ['2023-01-05', '2023-01-10', '2023-02-15', '2023-02-20'],
        'categoria': ['Electrónica', 'Electrónica', 'Accesorios', 'Accesorios'],
        'total': [100.0, 200.0, 50.0, 75.0]
    }
    return pd.DataFrame(data)

def test_agregar_ventas_estructura(df_ventas_test):
    """Test que verifica la estructura del DataFrame resultante."""
    resultado = agregar_ventas_por_categoria_y_mes(df_ventas_test)
    
    # Verificamos que el DataFrame tenga las columnas esperadas
    assert set(resultado.columns) == {'categoria', 'mes', 'total'}
    
    # Verificamos que el número de filas sea correcto (2 categorías x 2 meses = 4 filas)
    assert len(resultado) == 3  # Solo hay 3 combinaciones únicas de categoría-mes en los datos de prueba

def test_agregar_ventas_valores(df_ventas_test):
    """Test que verifica los valores agregados."""
    resultado = agregar_ventas_por_categoria_y_mes(df_ventas_test)
    
    # Verificamos los valores agregados para cada combinación de categoría y mes
    # Electrónica, 2023-01: 100 + 200 = 300
    electronica_enero = resultado[(resultado['categoria'] == 'Electrónica') & (resultado['mes'] == '2023-01')]
    assert len(electronica_enero) == 1
    assert electronica_enero.iloc[0]['total'] == 300.0
    
    # Accesorios, 2023-02: 50 + 75 = 125
    accesorios_febrero = resultado[(resultado['categoria'] == 'Accesorios') & (resultado['mes'] == '2023-02')]
    assert len(accesorios_febrero) == 1
    assert accesorios_febrero.iloc[0]['total'] == 125.0

def test_agregar_ventas_ordenamiento(df_ventas_test):
    """Test que verifica que el resultado esté ordenado por categoría y mes."""
    resultado = agregar_ventas_por_categoria_y_mes(df_ventas_test)
    
    # Convertimos a lista para verificar el orden
    categorias = resultado['categoria'].tolist()
    meses = resultado['mes'].tolist()
    
    # Verificamos que las categorías estén ordenadas alfabéticamente
    assert categorias == sorted(categorias)
    
    # Verificamos que dentro de cada categoría, los meses estén ordenados cronológicamente
    for categoria in set(categorias):
        meses_categoria = resultado[resultado['categoria'] == categoria]['mes'].tolist()
        assert meses_categoria == sorted(meses_categoria)

In [ ]:
# Ejecutamos los tests
!pytest -xvs test_agregacion.py

## Ejemplo 2: Testing de Transformaciones de Datos

En Data Engineering, a menudo necesitamos transformar datos de un formato a otro. Vamos a crear una función que transforme nuestro dataset de ventas a un formato "ancho" (wide format) para análisis, y luego escribiremos tests para verificar esta transformación.

In [ ]:
def transformar_a_formato_ancho(df):
    """Transforma el DataFrame de ventas a un formato ancho por categoría y mes.
    
    Args:
        df: DataFrame con columnas 'fecha', 'categoria' y 'total'
        
    Returns:
        DataFrame: DataFrame en formato ancho con meses como columnas y categorías como filas
    """
    # Primero agregamos por categoría y mes
    df_agregado = agregar_ventas_por_categoria_y_mes(df)
    
    # Pivotamos para obtener el formato ancho
    df_ancho = df_agregado.pivot(index='categoria', columns='mes', values='total')
    
    # Rellenamos los valores nulos con ceros
    df_ancho = df_ancho.fillna(0)
    
    # Añadimos una columna de total por categoría
    df_ancho['Total'] = df_ancho.sum(axis=1)
    
    return df_ancho

# Probamos la función
ventas_formato_ancho = transformar_a_formato_ancho(df_ventas)
ventas_formato_ancho.head()

Ahora, vamos a escribir tests para esta función:

In [ ]:
%%file ../tests/test_transformacion.py
import pytest
import pandas as pd
import numpy as np

def agregar_ventas_por_categoria_y_mes(df):
    """Agrega las ventas por categoría y mes."""
    df = df.copy()
    df['fecha'] = pd.to_datetime(df['fecha'])
    df['mes'] = df['fecha'].dt.strftime('%Y-%m')
    ventas_agregadas = df.groupby(['categoria', 'mes'])['total'].sum().reset_index()
    ventas_agregadas = ventas_agregadas.sort_values(['categoria', 'mes'])
    return ventas_agregadas

def transformar_a_formato_ancho(df):
    """Transforma el DataFrame de ventas a un formato ancho por categoría y mes."""
    df_agregado = agregar_ventas_por_categoria_y_mes(df)
    df_ancho = df_agregado.pivot(index='categoria', columns='mes', values='total')
    df_ancho = df_ancho.fillna(0)
    df_ancho['Total'] = df_ancho.sum(axis=1)
    return df_ancho

@pytest.fixture
def df_ventas_test():
    """Fixture que crea un DataFrame de prueba."""
    data = {
        'fecha': ['2023-01-05', '2023-01-10', '2023-02-15', '2023-02-20'],
        'categoria': ['Electrónica', 'Electrónica', 'Accesorios', 'Accesorios'],
        'total': [100.0, 200.0, 50.0, 75.0]
    }
    return pd.DataFrame(data)

def test_transformar_a_formato_ancho_estructura(df_ventas_test):
    """Test que verifica la estructura del DataFrame en formato ancho."""
    resultado = transformar_a_formato_ancho(df_ventas_test)
    
    # Verificamos que el índice contenga las categorías
    assert set(resultado.index) == {'Electrónica', 'Accesorios'}
    
    # Verificamos que las columnas incluyan los meses y el total
    assert set(resultado.columns) == {'2023-01', '2023-02', 'Total'}

def test_transformar_a_formato_ancho_valores(df_ventas_test):
    """Test que verifica los valores en el DataFrame en formato ancho."""
    resultado = transformar_a_formato_ancho(df_ventas_test)
    
    # Verificamos los valores para Electrónica
    assert resultado.loc['Electrónica', '2023-01'] == 300.0
    assert resultado.loc['Electrónica', '2023-02'] == 0.0  # No hay ventas en febrero
    assert resultado.loc['Electrónica', 'Total'] == 300.0
    
    # Verificamos los valores para Accesorios
    assert resultado.loc['Accesorios', '2023-01'] == 0.0  # No hay ventas en enero
    assert resultado.loc['Accesorios', '2023-02'] == 125.0
    assert resultado.loc['Accesorios', 'Total'] == 125.0

def test_transformar_a_formato_ancho_sin_nulos(df_ventas_test):
    """Test que verifica que no haya valores nulos en el resultado."""
    resultado = transformar_a_formato_ancho(df_ventas_test)
    
    # Verificamos que no haya valores nulos
    assert not resultado.isnull().any().any()

In [ ]:
# Ejecutamos los tests
!pytest -xvs test_transformacion.py

## Ejemplo 3: Testing de Validación de Datos con Esquemas

En Data Engineering, es común validar que los datos cumplan con un esquema específico. Vamos a crear una función que valide el esquema de nuestro dataset de ventas y escribiremos tests para verificar esta validación.

In [ ]:
def validar_esquema(df, esquema):
    """Valida que un DataFrame cumpla con un esquema específico.
    
    Args:
        df: DataFrame a validar
        esquema: Diccionario con columnas y sus tipos de datos esperados
        
    Returns:
        dict: Resultado de la validación
    """
    # Verificamos que todas las columnas requeridas estén presentes
    columnas_faltantes = [col for col in esquema.keys() if col not in df.columns]
    if columnas_faltantes:
        return {
            'valido': False,
            'error': f"Columnas faltantes: {', '.join(columnas_faltantes)}"
        }
    
    # Verificamos los tipos de datos
    tipos_incorrectos = {}
    for columna, tipo_esperado in esquema.items():
        # Para tipos numéricos, verificamos si son compatibles
        if tipo_esperado in ['int', 'float'] and pd.api.types.is_numeric_dtype(df[columna]):
            continue
        
        # Para fechas, verificamos si se pueden convertir
        if tipo_esperado == 'datetime' and pd.api.types.is_string_dtype(df[columna]):
            try:
                pd.to_datetime(df[columna])
                continue
            except:
                pass
        
        # Para strings, verificamos si son object o string
        if tipo_esperado == 'string' and pd.api.types.is_string_dtype(df[columna]):
            continue
        
        # Si llegamos aquí, el tipo no es compatible
        tipos_incorrectos[columna] = {
            'esperado': tipo_esperado,
            'actual': str(df[columna].dtype)
        }
    
    if tipos_incorrectos:
        return {
            'valido': False,
            'error': f"Tipos de datos incorrectos en {len(tipos_incorrectos)} columnas",
            'detalle': tipos_incorrectos
        }
    
    return {
        'valido': True,
        'mensaje': "El DataFrame cumple con el esquema especificado"
    }

# Definimos el esquema esperado para nuestro dataset de ventas
esquema_ventas = {
    'id': 'int',
    'fecha': 'datetime',
    'producto': 'string',
    'categoria': 'string',
    'precio': 'float',
    'cantidad': 'int',
    'descuento': 'float',
    'total': 'float'
}

# Probamos la función
resultado_validacion = validar_esquema(df_ventas, esquema_ventas)
resultado_validacion

Ahora, vamos a escribir tests para esta función:

In [ ]:
%%file ../tests/test_esquema.py
import pytest
import pandas as pd
import numpy as np

def validar_esquema(df, esquema):
    """Valida que un DataFrame cumpla con un esquema específico."""
    # Verificamos que todas las columnas requeridas estén presentes
    columnas_faltantes = [col for col in esquema.keys() if col not in df.columns]
    if columnas_faltantes:
        return {
            'valido': False,
            'error': f"Columnas faltantes: {', '.join(columnas_faltantes)}"
        }
    
    # Verificamos los tipos de datos
    tipos_incorrectos = {}
    for columna, tipo_esperado in esquema.items():
        # Para tipos numéricos, verificamos si son compatibles
        if tipo_esperado in ['int', 'float'] and pd.api.types.is_numeric_dtype(df[columna]):
            continue
        
        # Para fechas, verificamos si se pueden convertir
        if tipo_esperado == 'datetime' and pd.api.types.is_string_dtype(df[columna]):
            try:
                pd.to_datetime(df[columna])
                continue
            except:
                pass
        
        # Para strings, verificamos si son object o string
        if tipo_esperado == 'string' and pd.api.types.is_string_dtype(df[columna]):
            continue
        
        # Si llegamos aquí, el tipo no es compatible
        tipos_incorrectos[columna] = {
            'esperado': tipo_esperado,
            'actual': str(df[columna].dtype)
        }
    
    if tipos_incorrectos:
        return {
            'valido': False,
            'error': f"Tipos de datos incorrectos en {len(tipos_incorrectos)} columnas",
            'detalle': tipos_incorrectos
        }
    
    return {
        'valido': True,
        'mensaje': "El DataFrame cumple con el esquema especificado"
    }

@pytest.fixture
def df_valido():
    """Fixture que crea un DataFrame válido."""
    data = {
        'id': [1, 2, 3],
        'fecha': ['2023-01-05', '2023-01-10', '2023-01-15'],
        'producto': ['Laptop HP', 'Monitor Dell', 'Teclado Logitech'],
        'categoria': ['Electrónica', 'Electrónica', 'Accesorios'],
        'precio': [899.99, 249.99, 59.99],
        'cantidad': [1, 2, 3],
        'descuento': [0.05, 0.00, 0.10],
        'total': [854.99, 499.98, 161.97]
    }
    return pd.DataFrame(data)

@pytest.fixture
def df_columnas_faltantes():
    """Fixture que crea un DataFrame con columnas faltantes."""
    data = {
        'id': [1, 2, 3],
        'fecha': ['2023-01-05', '2023-01-10', '2023-01-15'],
        'producto': ['Laptop HP', 'Monitor Dell', 'Teclado Logitech'],
        # Falta 'categoria'
        'precio': [899.99, 249.99, 59.99],
        'cantidad': [1, 2, 3],
        # Falta 'descuento'
        'total': [854.99, 499.98, 161.97]
    }
    return pd.DataFrame(data)

@pytest.fixture
def df_tipos_incorrectos():
    """Fixture que crea un DataFrame con tipos de datos incorrectos."""
    data = {
        'id': ['1', '2', '3'],  # String en lugar de int
        'fecha': ['2023-01-05', '2023-01-10', '2023-01-15'],
        'producto': ['Laptop HP', 'Monitor Dell', 'Teclado Logitech'],
        'categoria': ['Electrónica', 'Electrónica', 'Accesorios'],
        'precio': [899.99, 249.99, 59.99],
        'cantidad': [1, 2, 3],
        'descuento': [0.05, 0.00, 0.10],
        'total': ['854.99', '499.98', '161.97']  # String en lugar de float
    }
    return pd.DataFrame(data)

def test_validar_esquema_df_valido(df_valido):
    """Test para validar un DataFrame que cumple con el esquema."""
    esquema = {
        'id': 'int',
        'fecha': 'datetime',
        'producto': 'string',
        'categoria': 'string',
        'precio': 'float',
        'cantidad': 'int',
        'descuento': 'float',
        'total': 'float'
    }
    resultado = validar_esquema(df_valido, esquema)
    assert resultado['valido'] is True
    assert 'mensaje' in resultado

def test_validar_esquema_columnas_faltantes(df_columnas_faltantes):
    """Test para validar un DataFrame con columnas faltantes."""
    esquema = {
        'id': 'int',
        'fecha': 'datetime',
        'producto': 'string',
        'categoria': 'string',  # Esta columna falta en el DataFrame
        'precio': 'float',
        'cantidad': 'int',
        'descuento': 'float',  # Esta columna falta en el DataFrame
        'total': 'float'
    }
    resultado = validar_esquema(df_columnas_faltantes, esquema)
    assert resultado['valido'] is False
    assert 'error' in resultado
    assert 'categoria' in resultado['error']
    assert 'descuento' in resultado['error']

def test_validar_esquema_tipos_incorrectos(df_tipos_incorrectos):
    """Test para validar un DataFrame con tipos de datos incorrectos."""
    esquema = {
        'id': 'int',  # En df_tipos_incorrectos, id es string
        'fecha': 'datetime',
        'producto': 'string',
        'categoria': 'string',
        'precio': 'float',
        'cantidad': 'int',
        'descuento': 'float',
        'total': 'float'  # En df_tipos_incorrectos, total es string
    }
    resultado = validar_esquema(df_tipos_incorrectos, esquema)
    assert resultado['valido'] is False
    assert 'error' in resultado
    assert 'detalle' in resultado
    assert 'id' in resultado['detalle']
    assert 'total' in resultado['detalle']

In [ ]:
# Ejecutamos los tests
!pytest -xvs test_esquema.py

## Ejemplo 4: Testing de Funciones de Limpieza de Datos

La limpieza de datos es una tarea común en Data Engineering. Vamos a crear una función que limpie nuestro dataset de ventas (eliminando duplicados, corrigiendo formatos, etc.) y escribiremos tests para verificar esta limpieza.

In [ ]:
def limpiar_dataset_ventas(df):
    """Limpia el dataset de ventas.
    
    Args:
        df: DataFrame de ventas
        
    Returns:
        DataFrame: DataFrame limpio
    """
    # Creamos una copia para no modificar el original
    df_limpio = df.copy()
    
    # 1. Convertimos la columna fecha a datetime
    df_limpio['fecha'] = pd.to_datetime(df_limpio['fecha'])
    
    # 2. Convertimos las columnas numéricas al tipo correcto
    df_limpio['precio'] = pd.to_numeric(df_limpio['precio'])
    df_limpio['cantidad'] = pd.to_numeric(df_limpio['cantidad']).astype(int)
    df_limpio['descuento'] = pd.to_numeric(df_limpio['descuento'])
    df_limpio['total'] = pd.to_numeric(df_limpio['total'])
    
    # 3. Eliminamos duplicados basados en todas las columnas
    df_limpio = df_limpio.drop_duplicates()
    
    # 4. Verificamos que el total sea correcto (precio * cantidad * (1 - descuento))
    df_limpio['total_calculado'] = df_limpio['precio'] * df_limpio['cantidad'] * (1 - df_limpio['descuento'])
    
    # 5. Corregimos los totales que difieren más de 0.01 del calculado
    diferencia = abs(df_limpio['total'] - df_limpio['total_calculado'])
    df_limpio.loc[diferencia > 0.01, 'total'] = df_limpio.loc[diferencia > 0.01, 'total_calculado']
    
    # 6. Eliminamos la columna temporal
    df_limpio = df_limpio.drop(columns=['total_calculado'])
    
    # 7. Normalizamos las categorías (primera letra mayúscula, resto minúsculas)
    df_limpio['categoria'] = df_limpio['categoria'].str.capitalize()
    
    return df_limpio

# Probamos la función
df_ventas_limpio = limpiar_dataset_ventas(df_ventas)
df_ventas_limpio.head()

Ahora, vamos a escribir tests para esta función:

In [ ]:
%%file ../tests/test_limpieza.py
import pytest
import pandas as pd
import numpy as np

def limpiar_dataset_ventas(df):
    """Limpia el dataset de ventas."""
    df_limpio = df.copy()
    df_limpio['fecha'] = pd.to_datetime(df_limpio['fecha'])
    df_limpio['precio'] = pd.to_numeric(df_limpio['precio'])
    df_limpio['cantidad'] = pd.to_numeric(df_limpio['cantidad']).astype(int)
    df_limpio['descuento'] = pd.to_numeric(df_limpio['descuento'])
    df_limpio['total'] = pd.to_numeric(df_limpio['total'])
    df_limpio = df_limpio.drop_duplicates()
    df_limpio['total_calculado'] = df_limpio['precio'] * df_limpio['cantidad'] * (1 - df_limpio['descuento'])
    diferencia = abs(df_limpio['total'] - df_limpio['total_calculado'])
    df_limpio.loc[diferencia > 0.01, 'total'] = df_limpio.loc[diferencia > 0.01, 'total_calculado']
    df_limpio = df_limpio.drop(columns=['total_calculado'])
    df_limpio['categoria'] = df_limpio['categoria'].str.capitalize()
    return df_limpio

@pytest.fixture
def df_con_problemas():
    """Fixture que crea un DataFrame con problemas para limpiar."""
    data = {
        'id': [1, 2, 3, 3],  # ID duplicado
        'fecha': ['2023-01-05', '2023-01-10', '2023-01-15', '2023-01-15'],
        'producto': ['Laptop HP', 'Monitor Dell', 'Teclado Logitech', 'Teclado Logitech'],
        'categoria': ['ELECTRÓNICA', 'electrónica', 'Accesorios', 'accesorios'],  # Inconsistencia en mayúsculas/minúsculas
        'precio': [899.99, 249.99, 59.99, 59.99],
        'cantidad': [1, 2, 3, 3],
        'descuento': [0.05, 0.00, 0.10, 0.10],
        'total': [854.99, 499.98, 161.97, 200.00]  # Total incorrecto en la última fila
    }
    return pd.DataFrame(data)

def test_limpiar_dataset_elimina_duplicados(df_con_problemas):
    """Test que verifica que la función elimine duplicados."""
    resultado = limpiar_dataset_ventas(df_con_problemas)
    
    # Verificamos que se hayan eliminado los duplicados
    assert len(resultado) == 3  # El DataFrame original tiene 4 filas, una duplicada
    assert resultado['id'].nunique() == 3  # Debe haber 3 IDs únicos

def test_limpiar_dataset_corrige_totales(df_con_problemas):
    """Test que verifica que la función corrija los totales incorrectos."""
    resultado = limpiar_dataset_ventas(df_con_problemas)
    
    # Calculamos los totales esperados
    totales_esperados = df_con_problemas['precio'] * df_con_problemas['cantidad'] * (1 - df_con_problemas['descuento'])
    
    # Verificamos que los totales se hayan corregido
    for i, row in resultado.iterrows():
        total_calculado = row['precio'] * row['cantidad'] * (1 - row['descuento'])
        assert abs(row['total'] - total_calculado) < 0.01, f"Total incorrecto en la fila {i}"

def test_limpiar_dataset_normaliza_categorias(df_con_problemas):
    """Test que verifica que la función normalice las categorías."""
    resultado = limpiar_dataset_ventas(df_con_problemas)
    
    # Verificamos que las categorías estén normalizadas (primera letra mayúscula, resto minúsculas)
    categorias_esperadas = {'Electrónica', 'Accesorios'}
    assert set(resultado['categoria'].unique()) == categorias_esperadas

def test_limpiar_dataset_convierte_tipos(df_con_problemas):
    """Test que verifica que la función convierta los tipos de datos correctamente."""
    resultado = limpiar_dataset_ventas(df_con_problemas)
    
    # Verificamos los tipos de datos
    assert pd.api.types.is_datetime64_dtype(resultado['fecha'])
    assert pd.api.types.is_float_dtype(resultado['precio'])
    assert pd.api.types.is_integer_dtype(resultado['cantidad'])
    assert pd.api.types.is_float_dtype(resultado['descuento'])
    assert pd.api.types.is_float_dtype(resultado['total'])

In [ ]:
# Ejecutamos los tests
!pytest -xvs test_limpieza.py

## Ejemplo 5: Testing de Integración para un Pipeline de ETL

En Data Engineering, a menudo trabajamos con pipelines de ETL (Extract, Transform, Load). Vamos a crear un pequeño pipeline de ETL para nuestro dataset de ventas y escribiremos tests de integración para verificar que todo el pipeline funcione correctamente.

In [ ]:
class PipelineETL:
    """Clase que implementa un pipeline de ETL para el dataset de ventas."""
    
    def __init__(self):
        """Inicializa el pipeline."""
        self.datos_originales = None
        self.datos_limpios = None
        self.datos_transformados = None
        self.metricas = None
    
    def extraer(self, ruta_archivo):
        """Extrae los datos del archivo CSV.
        
        Args:
            ruta_archivo: Ruta al archivo CSV
            
        Returns:
            self: Para encadenamiento de métodos
        """
        self.datos_originales = pd.read_csv(ruta_archivo)
        return self
    
    def transformar(self):
        """Transforma los datos (limpieza y agregación).
        
        Returns:
            self: Para encadenamiento de métodos
        """
        if self.datos_originales is None:
            raise ValueError("No hay datos para transformar. Ejecuta extraer() primero.")
        
        # Limpiamos los datos
        self.datos_limpios = limpiar_dataset_ventas(self.datos_originales)
        
        # Transformamos a formato ancho por categoría y mes
        self.datos_transformados = transformar_a_formato_ancho(self.datos_limpios)
        
        # Calculamos métricas
        self.metricas = calcular_metricas_ventas(self.datos_limpios)
        
        return self
    
    def cargar(self, directorio_salida):
        """Carga los datos transformados en archivos de salida.
        
        Args:
            directorio_salida: Directorio donde guardar los archivos
            
        Returns:
            dict: Rutas de los archivos generados
        """
        if self.datos_transformados is None or self.metricas is None:
            raise ValueError("No hay datos transformados. Ejecuta transformar() primero.")
        
        # Creamos el directorio si no existe
        os.makedirs(directorio_salida, exist_ok=True)
        
        # Guardamos los datos limpios
        ruta_datos_limpios = os.path.join(directorio_salida, 'ventas_limpias.csv')
        self.datos_limpios.to_csv(ruta_datos_limpios, index=False)
        
        # Guardamos los datos transformados
        ruta_datos_transformados = os.path.join(directorio_salida, 'ventas_por_categoria_mes.csv')
        self.datos_transformados.to_csv(ruta_datos_transformados)
        
        # Guardamos las métricas
        ruta_metricas = os.path.join(directorio_salida, 'metricas_ventas.json')
        with open(ruta_metricas, 'w') as f:
            import json
            json.dump(self.metricas, f, indent=4)
        
        return {
            'datos_limpios': ruta_datos_limpios,
            'datos_transformados': ruta_datos_transformados,
            'metricas': ruta_metricas
        }
    
    def ejecutar_pipeline(self, ruta_archivo, directorio_salida):
        """Ejecuta todo el pipeline de ETL.
        
        Args:
            ruta_archivo: Ruta al archivo CSV de entrada
            directorio_salida: Directorio donde guardar los archivos de salida
            
        Returns:
            dict: Rutas de los archivos generados
        """
        return self.extraer(ruta_archivo).transformar().cargar(directorio_salida)

# Probamos el pipeline
pipeline = PipelineETL()
rutas_archivos = pipeline.ejecutar_pipeline('../data/ventas_productos.csv', '../output')
rutas_archivos

Ahora, vamos a escribir tests de integración para nuestro pipeline de ETL:

In [ ]:
%%file ../tests/test_pipeline_etl.py
import pytest
import pandas as pd
import numpy as np
import os
import json
import tempfile
import shutil

# Importamos las funciones necesarias
def limpiar_dataset_ventas(df):
    """Limpia el dataset de ventas."""
    df_limpio = df.copy()
    df_limpio['fecha'] = pd.to_datetime(df_limpio['fecha'])
    df_limpio['precio'] = pd.to_numeric(df_limpio['precio'])
    df_limpio['cantidad'] = pd.to_numeric(df_limpio['cantidad']).astype(int)
    df_limpio['descuento'] = pd.to_numeric(df_limpio['descuento'])
    df_limpio['total'] = pd.to_numeric(df_limpio['total'])
    df_limpio = df_limpio.drop_duplicates()
    df_limpio['total_calculado'] = df_limpio['precio'] * df_limpio['cantidad'] * (1 - df_limpio['descuento'])
    diferencia = abs(df_limpio['total'] - df_limpio['total_calculado'])
    df_limpio.loc[diferencia > 0.01, 'total'] = df_limpio.loc[diferencia > 0.01, 'total_calculado']
    df_limpio = df_limpio.drop(columns=['total_calculado'])
    df_limpio['categoria'] = df_limpio['categoria'].str.capitalize()
    return df_limpio

def agregar_ventas_por_categoria_y_mes(df):
    """Agrega las ventas por categoría y mes."""
    df = df.copy()
    df['fecha'] = pd.to_datetime(df['fecha'])
    df['mes'] = df['fecha'].dt.strftime('%Y-%m')
    ventas_agregadas = df.groupby(['categoria', 'mes'])['total'].sum().reset_index()
    ventas_agregadas = ventas_agregadas.sort_values(['categoria', 'mes'])
    return ventas_agregadas

def transformar_a_formato_ancho(df):
    """Transforma el DataFrame de ventas a un formato ancho por categoría y mes."""
    df_agregado = agregar_ventas_por_categoria_y_mes(df)
    df_ancho = df_agregado.pivot(index='categoria', columns='mes', values='total')
    df_ancho = df_ancho.fillna(0)
    df_ancho['Total'] = df_ancho.sum(axis=1)
    return df_ancho

def calcular_metricas_ventas(df):
    """Calcula métricas de ventas a partir de un DataFrame de ventas."""
    metricas = {
        'total_ventas': float(df['total'].sum()),
        'promedio_venta': float(df['total'].mean()),
        'total_productos_vendidos': int(df['cantidad'].sum()),
        'precio_promedio': float(df['precio'].mean()),
        'descuento_promedio': float(df['descuento'].mean()),
        'ahorro_total': float((df['precio'] * df['cantidad'] * df['descuento']).sum())
    }
    return metricas

class PipelineETL:
    """Clase que implementa un pipeline de ETL para el dataset de ventas."""
    
    def __init__(self):
        """Inicializa el pipeline."""
        self.datos_originales = None
        self.datos_limpios = None
        self.datos_transformados = None
        self.metricas = None
    
    def extraer(self, ruta_archivo):
        """Extrae los datos del archivo CSV."""
        self.datos_originales = pd.read_csv(ruta_archivo)
        return self
    
    def transformar(self):
        """Transforma los datos (limpieza y agregación)."""
        if self.datos_originales is None:
            raise ValueError("No hay datos para transformar. Ejecuta extraer() primero.")
        
        # Limpiamos los datos
        self.datos_limpios = limpiar_dataset_ventas(self.datos_originales)
        
        # Transformamos a formato ancho por categoría y mes
        self.datos_transformados = transformar_a_formato_ancho(self.datos_limpios)
        
        # Calculamos métricas
        self.metricas = calcular_metricas_ventas(self.datos_limpios)
        
        return self
    
    def cargar(self, directorio_salida):
        """Carga los datos transformados en archivos de salida."""
        if self.datos_transformados is None or self.metricas is None:
            raise ValueError("No hay datos transformados. Ejecuta transformar() primero.")
        
        # Creamos el directorio si no existe
        os.makedirs(directorio_salida, exist_ok=True)
        
        # Guardamos los datos limpios
        ruta_datos_limpios = os.path.join(directorio_salida, 'ventas_limpias.csv')
        self.datos_limpios.to_csv(ruta_datos_limpios, index=False)
        
        # Guardamos los datos transformados
        ruta_datos_transformados = os.path.join(directorio_salida, 'ventas_por_categoria_mes.csv')
        self.datos_transformados.to_csv(ruta_datos_transformados)
        
        # Guardamos las métricas
        ruta_metricas = os.path.join(directorio_salida, 'metricas_ventas.json')
        with open(ruta_metricas, 'w') as f:
            json.dump(self.metricas, f, indent=4)
        
        return {
            'datos_limpios': ruta_datos_limpios,
            'datos_transformados': ruta_datos_transformados,
            'metricas': ruta_metricas
        }
    
    def ejecutar_pipeline(self, ruta_archivo, directorio_salida):
        """Ejecuta todo el pipeline de ETL."""
        return self.extraer(ruta_archivo).transformar().cargar(directorio_salida)

@pytest.fixture
def datos_prueba():
    """Fixture que crea un archivo CSV temporal con datos de prueba."""
    # Creamos un directorio temporal
    directorio_temp = tempfile.mkdtemp()
    
    # Creamos datos de prueba
    data = {
        'id': [1, 2, 3, 4],
        'fecha': ['2023-01-05', '2023-01-10', '2023-02-15', '2023-02-20'],
        'producto': ['Laptop HP', 'Monitor Dell', 'Teclado Logitech', 'Mouse Inalámbrico'],
        'categoria': ['Electrónica', 'Electrónica', 'Accesorios', 'Accesorios'],
        'precio': [899.99, 249.99, 59.99, 29.99],
        'cantidad': [1, 2, 3, 5],
        'descuento': [0.05, 0.00, 0.10, 0.00],
        'total': [854.99, 499.98, 161.97, 149.95]
    }
    df = pd.DataFrame(data)
    
    # Guardamos en un archivo CSV
    ruta_csv = os.path.join(directorio_temp, 'ventas_test.csv')
    df.to_csv(ruta_csv, index=False)
    
    # Creamos un directorio para los archivos de salida
    directorio_salida = os.path.join(directorio_temp, 'output')
    os.makedirs(directorio_salida, exist_ok=True)
    
    # Devolvemos las rutas
    yield {
        'ruta_csv': ruta_csv,
        'directorio_salida': directorio_salida,
        'directorio_temp': directorio_temp
    }
    
    # Limpiamos después de las pruebas
    shutil.rmtree(directorio_temp)

@pytest.mark.integracion
def test_pipeline_etl_completo(datos_prueba):
    """Test de integración para el pipeline ETL completo."""
    # Ejecutamos el pipeline
    pipeline = PipelineETL()
    rutas_archivos = pipeline.ejecutar_pipeline(
        datos_prueba['ruta_csv'],
        datos_prueba['directorio_salida']
    )
    
    # Verificamos que se hayan generado los archivos esperados
    assert os.path.exists(rutas_archivos['datos_limpios'])
    assert os.path.exists(rutas_archivos['datos_transformados'])
    assert os.path.exists(rutas_archivos['metricas'])
    
    # Verificamos el contenido de los archivos
    # 1. Datos limpios
    df_limpios = pd.read_csv(rutas_archivos['datos_limpios'])
    assert len(df_limpios) == 4  # No debería haber duplicados
    assert 'id' in df_limpios.columns
    assert 'fecha' in df_limpios.columns
    
    # 2. Datos transformados
    df_transformados = pd.read_csv(rutas_archivos['datos_transformados'])
    assert 'categoria' in df_transformados.columns
    assert 'Total' in df_transformados.columns
    
    # 3. Métricas
    with open(rutas_archivos['metricas'], 'r') as f:
        metricas = json.load(f)
    assert 'total_ventas' in metricas
    assert 'promedio_venta' in metricas
    assert 'total_productos_vendidos' in metricas

@pytest.mark.integracion
def test_pipeline_etl_valores(datos_prueba):
    """Test de integración que verifica los valores generados por el pipeline."""
    # Ejecutamos el pipeline
    pipeline = PipelineETL()
    rutas_archivos = pipeline.ejecutar_pipeline(
        datos_prueba['ruta_csv'],
        datos_prueba['directorio_salida']
    )
    
    # Verificamos las métricas
    with open(rutas_archivos['metricas'], 'r') as f:
        metricas = json.load(f)
    
    # La suma de los totales debe ser 854.99 + 499.98 + 161.97 + 149.95 = 1666.89
    assert abs(metricas['total_ventas'] - 1666.89) < 0.01
    
    # El total de productos vendidos debe ser 1 + 2 + 3 + 5 = 11
    assert metricas['total_productos_vendidos'] == 11
    
    # Verificamos los datos transformados
    df_transformados = pd.read_csv(rutas_archivos['datos_transformados'], index_col=0)
    
    # Debe haber 2 categorías
    assert len(df_transformados) == 2
    
    # La categoría 'Accesorios' debe tener un total de 161.97 + 149.95 = 311.92
    assert abs(df_transformados.loc['Accesorios', 'Total'] - 311.92) < 0.01
    
    # La categoría 'Electrónica' debe tener un total de 854.99 + 499.98 = 1354.97
    assert abs(df_transformados.loc['Electrónica', 'Total'] - 1354.97) < 0.01

In [ ]:
# Ejecutamos los tests de integración
!pytest -xvs test_pipeline_etl.py -m integracion

## Ejemplo 6: Testing de Rendimiento

En Data Engineering, el rendimiento es a menudo una preocupación importante, especialmente cuando trabajamos con grandes conjuntos de datos. Vamos a crear tests de rendimiento para medir el tiempo de ejecución de nuestras funciones.

In [ ]:
%%file ../tests/test_rendimiento.py
import pytest
import pandas as pd
import numpy as np
import time
import os

# Importamos las funciones a testear
def limpiar_dataset_ventas(df):
    """Limpia el dataset de ventas."""
    df_limpio = df.copy()
    df_limpio['fecha'] = pd.to_datetime(df_limpio['fecha'])
    df_limpio['precio'] = pd.to_numeric(df_limpio['precio'])
    df_limpio['cantidad'] = pd.to_numeric(df_limpio['cantidad']).astype(int)
    df_limpio['descuento'] = pd.to_numeric(df_limpio['descuento'])
    df_limpio['total'] = pd.to_numeric(df_limpio['total'])
    df_limpio = df_limpio.drop_duplicates()
    df_limpio['total_calculado'] = df_limpio['precio'] * df_limpio['cantidad'] * (1 - df_limpio['descuento'])
    diferencia = abs(df_limpio['total'] - df_limpio['total_calculado'])
    df_limpio.loc[diferencia > 0.01, 'total'] = df_limpio.loc[diferencia > 0.01, 'total_calculado']
    df_limpio = df_limpio.drop(columns=['total_calculado'])
    df_limpio['categoria'] = df_limpio['categoria'].str.capitalize()
    return df_limpio

def agregar_ventas_por_categoria_y_mes(df):
    """Agrega las ventas por categoría y mes."""
    df = df.copy()
    df['fecha'] = pd.to_datetime(df['fecha'])
    df['mes'] = df['fecha'].dt.strftime('%Y-%m')
    ventas_agregadas = df.groupby(['categoria', 'mes'])['total'].sum().reset_index()
    ventas_agregadas = ventas_agregadas.sort_values(['categoria', 'mes'])
    return ventas_agregadas

def transformar_a_formato_ancho(df):
    """Transforma el DataFrame de ventas a un formato ancho por categoría y mes."""
    df_agregado = agregar_ventas_por_categoria_y_mes(df)
    df_ancho = df_agregado.pivot(index='categoria', columns='mes', values='total')
    df_ancho = df_ancho.fillna(0)
    df_ancho['Total'] = df_ancho.sum(axis=1)
    return df_ancho

@pytest.fixture
def df_ventas_pequeno():
    """Fixture que crea un DataFrame pequeño (20 filas)."""
    # Cargamos el dataset original
    ruta_csv = os.path.join(os.path.dirname(os.path.dirname(__file__)), 'data', 'ventas_productos.csv')
    return pd.read_csv(ruta_csv)

@pytest.fixture
def df_ventas_mediano():
    """Fixture que crea un DataFrame mediano (200 filas)."""
    # Cargamos el dataset original y lo replicamos 10 veces
    ruta_csv = os.path.join(os.path.dirname(os.path.dirname(__file__)), 'data', 'ventas_productos.csv')
    df_original = pd.read_csv(ruta_csv)
    return pd.concat([df_original] * 10, ignore_index=True)

@pytest.fixture
def df_ventas_grande():
    """Fixture que crea un DataFrame grande (2000 filas)."""
    # Cargamos el dataset original y lo replicamos 100 veces
    ruta_csv = os.path.join(os.path.dirname(os.path.dirname(__file__)), 'data', 'ventas_productos.csv')
    df_original = pd.read_csv(ruta_csv)
    return pd.concat([df_original] * 100, ignore_index=True)

@pytest.mark.rendimiento
def test_rendimiento_limpiar_dataset(df_ventas_pequeno, df_ventas_mediano, df_ventas_grande):
    """Test de rendimiento para la función de limpieza de datos."""
    # Medimos el tiempo para el dataset pequeño
    inicio = time.time()
    limpiar_dataset_ventas(df_ventas_pequeno)
    tiempo_pequeno = time.time() - inicio
    
    # Medimos el tiempo para el dataset mediano
    inicio = time.time()
    limpiar_dataset_ventas(df_ventas_mediano)
    tiempo_mediano = time.time() - inicio
    
    # Medimos el tiempo para el dataset grande
    inicio = time.time()
    limpiar_dataset_ventas(df_ventas_grande)
    tiempo_grande = time.time() - inicio
    
    # Imprimimos los resultados
    print(f"\nTiempos de ejecución para limpiar_dataset_ventas:")
    print(f"Dataset pequeño (20 filas): {tiempo_pequeno:.4f} segundos")
    print(f"Dataset mediano (200 filas): {tiempo_mediano:.4f} segundos")
    print(f"Dataset grande (2000 filas): {tiempo_grande:.4f} segundos")
    
    # Verificamos que el tiempo de ejecución sea aproximadamente lineal
    # (el tiempo para el dataset grande debería ser aproximadamente 10 veces el tiempo para el dataset mediano)
    ratio = tiempo_grande / tiempo_mediano
    assert 5 < ratio < 15, f"La relación de tiempo grande/mediano es {ratio}, debería ser cercana a 10"

@pytest.mark.rendimiento
def test_rendimiento_transformar_a_formato_ancho(df_ventas_pequeno, df_ventas_mediano, df_ventas_grande):
    """Test de rendimiento para la función de transformación a formato ancho."""
    # Limpiamos los datasets primero
    df_pequeno_limpio = limpiar_dataset_ventas(df_ventas_pequeno)
    df_mediano_limpio = limpiar_dataset_ventas(df_ventas_mediano)
    df_grande_limpio = limpiar_dataset_ventas(df_ventas_grande)
    
    # Medimos el tiempo para el dataset pequeño
    inicio = time.time()
    transformar_a_formato_ancho(df_pequeno_limpio)
    tiempo_pequeno = time.time() - inicio
    
    # Medimos el tiempo para el dataset mediano
    inicio = time.time()
    transformar_a_formato_ancho(df_mediano_limpio)
    tiempo_mediano = time.time() - inicio
    
    # Medimos el tiempo para el dataset grande
    inicio = time.time()
    transformar_a_formato_ancho(df_grande_limpio)
    tiempo_grande = time.time() - inicio
    
    # Imprimimos los resultados
    print(f"\nTiempos de ejecución para transformar_a_formato_ancho:")
    print(f"Dataset pequeño (20 filas): {tiempo_pequeno:.4f} segundos")
    print(f"Dataset mediano (200 filas): {tiempo_mediano:.4f} segundos")
    print(f"Dataset grande (2000 filas): {tiempo_grande:.4f} segundos")
    
    # Verificamos que el tiempo de ejecución no sea excesivo
    assert tiempo_grande < 5, f"El tiempo para el dataset grande es {tiempo_grande:.4f} segundos, debería ser menor a 5 segundos"

In [ ]:
# Ejecutamos los tests de rendimiento
!pytest -xvs test_rendimiento.py -m rendimiento

## Conclusión

En este notebook, hemos explorado varios ejemplos prácticos de cómo utilizar pytest en proyectos de Data Engineering. Hemos visto:

1. **Testing de funciones de agregación**: Verificamos que nuestras funciones de agregación produzcan los resultados esperados.
2. **Testing de transformaciones de datos**: Aseguramos que nuestras transformaciones de datos funcionen correctamente.
3. **Testing de validación de datos con esquemas**: Verificamos que nuestros datos cumplan con esquemas específicos.
4. **Testing de funciones de limpieza de datos**: Aseguramos que nuestras funciones de limpieza corrijan problemas comunes en los datos.
5. **Testing de integración para un pipeline de ETL**: Verificamos que todo el pipeline de ETL funcione correctamente de principio a fin.
6. **Testing de rendimiento**: Medimos y verificamos el rendimiento de nuestras funciones con diferentes tamaños de datos.

Estos ejemplos ilustran cómo pytest puede ayudarnos a asegurar la calidad y confiabilidad de nuestros proyectos de Data Engineering. Al implementar tests automatizados, podemos detectar problemas temprano, refactorizar con confianza y garantizar que nuestros pipelines de datos produzcan resultados correctos y consistentes.

En el siguiente notebook, veremos ejercicios prácticos para que puedas aplicar lo aprendido y desarrollar tus habilidades de testing en Data Engineering.